In [1]:
import os
import ProgressBar as pb
from pymail import alert

n = 10000 # number of simulations in experiment

def date2DOY(s):
    return int(pd.datetime.strftime(s,'%j'))

## P301

## Nr1

In [2]:
# change leaf on awk script

site = 'Nr1'
outsite = 'Nwt'
method = '7'
loc = 'ComoCreek_new'

clim = 'boulder_crn_corr'

# daily climate base stations
daily_clim = 'boulder_crn_corr_basestation'
daily_clim_tmp = 'boulder_crn_corr_tmp'

# hourly climate basestations
hourly_clim = 'niwot_snowmelt_ex_basestation'
hourly_clim_tmp = 'niwot_snowmelt_tmp'

# simulation components
whdr = './worldfiles/world.AFX.sm.hdr' # worldfile header
t = './tecfiles/snow.tec' # start date
f = './flowtables/AFX.flow' # end date
strt = '2009 10 1 1' # tecfile
nd = '2010 10 1 1' # flowtable 

gw1 = 0.624
gw2 = 0.21
lai = 0.624

## Vcm

In [3]:
soils = pd.read_pickle('./data/soils_%s_smex%s.pcl'%(site,method))

In [4]:
smex = pd.read_pickle('./data/params_%s_smex_method%s.pcl'%(site,method))

In [5]:
smex['DOY_leafon'] = smex.date_peakSWE.map(date2DOY) # create day of year peak swe

In [6]:
smex.head()

,SMR,TpeakSWE,peakSWE,ablSeas,Tnosnow,ref,date_ONS,DOAS,date_AS,idx,date_peakSWE,temp_out,MAP,DOY_leafon
114927,0.007447,180.0,0.413660,56.0,236.0,2010,2010-05-24,15,2009-10-15,0,2010-03-29,True,6.165935,88
90071,0.017967,202.0,0.423372,24.0,226.0,2010,2010-05-14,15,2009-10-15,1,2010-04-20,False,6.178629,110
55307,0.014164,236.0,0.470056,34.0,270.0,2010,2010-06-27,15,2009-10-15,2,2010-05-24,False,6.099164,144
13817,0.013006,198.0,0.478686,37.0,235.0,2010,2010-05-23,15,2009-10-15,3,2010-04-16,False,6.165935,106
71193,0.014850,235.0,0.410855,28.0,263.0,2010,2010-06-20,15,2009-10-15,4,2010-05-23,False,6.099164,143


In [7]:
# set the command for changing the leaf on date
if site == 'Vcm':
    leafoncmd = 'awk -f ./scripts/change_leafon.awk par=%s < ./defs2/veg_evergreen.def > ./defs2/veg_evergreen.tmp'

if site == 'Nr1':
    leafoncmd = 'awk -f ./scripts/change_leafon.awk par=%s < ../defs/veg_evergreen.def > ../defs/veg_evergreen.tmp'

if site == 'P301':
    leafoncmd = 'awk -f ./scripts/change_leafon.awk par=%s < ./defs/veg_evergreen.def > ./defs/veg_evergreen.tmp'

In [8]:
# make some lists and varaibles to generate the parameter data frame after the run
idx = 0
idxs = []
scenidxs = []
pas = []
pos = []
ms = []
ksats = []
combos = []
smrs = []
tpeakswes = []
peakswes = []
maps = []
refs = []

os.chdir('/RHESSys/%s/'%loc) # change the working directory to RHESSys

# copy the climate data

os.system('cp ./clim/%s.rain ./clim/smexperiment/smex%s/daily/'%(clim,method))
os.system('cp ./clim/%s.tmin ./clim/smexperiment/smex%s/daily/'%(clim,method))
os.system('cp ./clim/%s.tavg ./clim/smexperiment/smex%s/daily/'%(clim,method))
os.system('cp ./clim/%s.tmax ./clim/smexperiment/smex%s/daily/'%(clim,method))
os.system('cp ./clim/%s.daytime_rain_duration ./clim/smexperiment/smex%s/daily/'%(clim,method))
            
p = pb.ProgressBar(n)
for i in smex.idx: # loop through each scenario, 10,000 iterations here
    # pull out some parameters
    
    smr = smex.loc[smex.idx == i,'SMR'].as_matrix()[0]
    peakswe = smex.loc[smex.idx == i,'peakSWE'].as_matrix()[0]
    tpeakswe = smex.loc[smex.idx == i,'TpeakSWE'].as_matrix()[0]
    leafon = smex.loc[smex.idx == i,'DOY_leafon'].as_matrix()[0]
    ref = smex.loc[smex.idx == i,'ref'].as_matrix()[0]
    MAP = smex.loc[smex.idx == i,'MAP'].as_matrix()[0]
    
    
    # rename climate for each scenario
    tmp1 = 'awk -f ./scripts/change_daily_clim.awk par=%s < ./clim/%s > ./clim/%s'%(i,daily_clim,daily_clim_tmp)
    os.system(tmp1)
    tmp2 = 'awk -f ./scripts/change_hourly_clim.awk par=%s < ./clim/%s > ./clim/%s'%(i,hourly_clim,hourly_clim_tmp)
    os.system(tmp2)
    #tmp3 = 
    os.system('cp ./clim/smexperiment/smex%s/daily/%s.tavg ./clim/smexperiment/smex%s/daily/%s_%s.tavg'%(method,clim,method,clim,i))
    #tmp4 = 
    os.system('cp ./clim/smexperiment/smex%s/daily/%s.tmin ./clim/smexperiment/smex%s/daily/%s_%s.tmin'%(method,clim,method,clim,i))
    #tmp5 = 
    os.system('cp ./clim/smexperiment/smex%s/daily/%s.tmax ./clim/smexperiment/smex%s/daily/%s_%s.tmax'%(method,clim,method,clim,i))
    #tmp6 = 
    os.system('cp ./clim/smexperiment/smex%s/daily/%s.daytime_rain_duration ./clim/smexperiment/smex%s/daily/%s_%s.daytime_rain_duration'%(method,clim,method,clim,i))
    
    # change the leaf on date
    os.system(leafoncmd%leafon)    
    
    for combo in soils.combo: # iterate through the combination, 25 iterations here, run each scenario 25 times with different soil parameters
        w = './clim/smexperiment/smex%s/worldfiles/smex%s.%s'%(method,method,combo) # set worlfile
        
        pa_scalar = soils.loc[soils.combo == combo,'pa_scalar'].as_matrix()[0]
        po_scalar = soils.loc[soils.combo == combo,'po_scalar'].as_matrix()[0]
        m_scalar = soils.loc[soils.combo == combo,'m_scalar'].as_matrix()[0]
        ksat_scalar = soils.loc[soils.combo == combo,'ksat_scalar'].as_matrix()[0]
    
        cmd = 'rhessys5.19.test -w %s -whdr %s -t %s -b -p -s %s %s -sv %s %s -svalt %s %s -gw %s %s -vgsen %s 1 1 -pre /Volumes/data/RHESSys_out/%s/smex%s_sen/smex%s_sen_%s -st %s -ed %s -r %s'%(w,whdr,t,m_scalar,ksat_scalar,m_scalar,ksat_scalar,po_scalar,pa_scalar,gw1,gw2,lai,outsite,method,method,idx,strt,nd,f)
        
        os.system(cmd)
        
        # populate list and update counter
        idxs.append(i) # the snowmelt experiment number, this will be used to mate scenario parameters to the output, goes from 0 to 9,999
        pas.append(soils.loc[soils.combo == combo,'pa'].as_matrix()[0])
        pos.append(soils.loc[soils.combo == combo,'po'].as_matrix()[0])
        ms.append(soils.loc[soils.combo == combo,'m'].as_matrix()[0])
        ksats.append(soils.loc[soils.combo == combo,'ksat'].as_matrix()[0])
        combos.append(combo)
        scenidxs.append(idx) # append the scenario index number, this will be used to mate the output to the soil parameters used, goes from 0 to 249,999
        smrs.append(smr)
        tpeakswes.append(tpeakswe)
        peakswes.append(peakswe)
        maps.append(MAP)
        refs.append(ref)
        
        idx += 1     
    p.animate_ipython(i)

params = pd.DataFrame({'pa':pas,'po':pos,'m':ms,'ksat':ksats,'idx':idxs,'combo':combos,'scenidx':scenidxs,'SMR':smrs,
                       'TpeakSWE':tpeakswes,'PeakSWE':peakswes,'ref':refs,'MAP':maps})
os.chdir('/Volumes/Users/Theo/projects/RHESSys/smex/') # change directory back to normal

#params = pd.merge(left=params,right=smex,on='idx',how='left')

params.to_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))
    
alert.send_alert('barnhatb@colorado.edu','%s smex%s_sen finished'%(site,method),'Your script has finished')

[****************100%******************]  9999 of 10000 complete
